In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
from sqlalchemy import create_engine
import re
import requests as req
import json
import datetime
import time

In [2]:
KEY = 'b50bc5-5af08a'

BCN = 'BCN'

status = 'landed'

url = f"http://aviation-edge.com/v2/public/timetable?key={KEY}&iataCode={BCN}&type=departure&status={status}"


## Get flight information request

In [11]:
def get_data(airport,status = 'landed'):
    
    KEY = 'b50bc5-5af08a'
    url = f"http://aviation-edge.com/v2/public/timetable?key={KEY}&iataCode={airport}&type=departure&status={status}"    
    response = req.get(url)
    print(response)
    print((datetime.datetime.now()))
    json_file = response.json()

    if type(json_file) == list:
        df_f = pd.DataFrame(json_file)
        return df_f

    
    
    
    
    

In [12]:
df_barcelona = get_data(airport='BCN')

<Response [200]>
2019-07-25 14:40:33.470605


In [36]:
type(df_barcelona)

pandas.core.frame.DataFrame

In [15]:
def clean_data(data_df):
    data_df["airln_name"] = data_df["airline"].apply(lambda x: x['name'])
    data_df["airln_code"] = data_df["airline"].apply(lambda x: x['iataCode'])
    data_df['arri_delay'] = data_df['arrival'].apply(lambda x: x['delay'])
    data_df['arri_airpt'] = data_df['arrival'].apply(lambda x: x['iataCode'])
    data_df['schd_arri_time'] = data_df['arrival'].apply(lambda x: x['scheduledTime'])
    data_df['codeshared'] = data_df['codeshared'].apply(lambda x: x['airline']['name'] if x else None)
    data_df['depart_delay'] = data_df['departure'].apply(lambda x: x['delay'])
    data_df['depart_airpt'] = data_df['departure'].apply(lambda x: x['iataCode'])
    data_df['schd_depart_time'] = data_df['departure'].apply(lambda x: x['scheduledTime'])
    data_df['depart_gate'] = data_df['departure'].apply(lambda x: x['gate'])
    data_df['depart_terminal'] = data_df['departure'].apply(lambda x: x['terminal'])
    data_df['icaonumb'] = data_df['flight'].apply(lambda x: x['icaoNumber'])
    data_df['flight'] = data_df['flight'].apply(lambda x: x['number'])
    data_df = data_df.drop(['airline','arrival','departure'], axis =1)
    
    return data_df
    

## Get airports by country

In [16]:
def get_data_airpts(country):
    
    KEY = 'b50bc5-5af08a'
    url_airpt = f'https://aviation-edge.com/v2/public/airportDatabase?key={KEY}&codeIso2Country={country}'
    
    response = req.get(url_airpt)
    print(response)
    print((datetime.datetime.now()))
    json_file = response.json()
    df = pd.DataFrame(json_file)
    return df

In [17]:
spain_airpts = get_data_airpts('ES')

<Response [200]>
2019-07-25 14:43:19.685737


In [115]:
spain_airpts.head(2)

,GMT,airportId,codeIataAirport,codeIataCity,codeIcaoAirport,codeIso2Country,geonameId,latitudeAirport,longitudeAirport,nameAirport,nameCountry,phone,timezone
0,1,27,ABC,ABC,LEAB,ES,6299323,38.948334,-1.863333,Los Llanos,Spain,,Europe/Madrid
1,0,55,ACE,ACE,GCRR,ES,6297189,28.950668,-13.609059,Lanzarote,Spain,902 404 704,Atlantic/Canary


In [101]:
def get_iata_codes(df_airpts):
    airpts_lst = df_airpts['codeIataAirport'].to_list()
    country_code = df_airpts['codeIso2Country'].to_list()
    name = df_airpts['nameAirport'].to_list()
    return airpts_lst,country_code, name

### Spain

In [102]:
spain_airpts_lst, country_code_sp, name_airport = get_iata_codes(df_airpts=spain_airpts)


In [89]:
print(spain_airpts_lst)

['ABC', 'ACE', 'AEI', 'AGP', 'ALC', 'BCN', 'BIO', 'BJZ', 'CDT', 'CDZ', 'EAS', 'EEL', 'EEM', 'EER', 'EEU', 'FES', 'FGR', 'FUE', 'GEN', 'GGA', 'GMZ', 'GRO', 'GRX', 'HEV', 'HSK', 'IBZ', 'ILD', 'JCU', 'LCG', 'LEI', 'LEN', 'LEU', 'LPA', 'MAD', 'MAH', 'MJV', 'MLN', 'NDO', 'ODB', 'OVD', 'OZI', 'OZL', 'OZP', 'OZU', 'PMI', 'PNA', 'PXS', 'QGN', 'QLQ', 'REU', 'RGS', 'RJL', 'RMU', 'ROZ', 'RRA', 'SCQ', 'SDR', 'SLM', 'SPC', 'SPO', 'SVQ', 'TFN', 'TFS', 'TOJ', 'UER', 'UTL', 'VDE', 'VGO', 'VIT', 'VLC', 'VLL', 'XIV', 'XJB', 'XJI', 'XJJ', 'XJN', 'XJO', 'XJR', 'XJU', 'XOC', 'XOJ', 'XOU', 'XQA', 'XRY', 'XTI', 'XTJ', 'XUA', 'XUF', 'XUT', 'XZZ', 'YJB', 'YJC', 'YJD', 'YJE', 'YJG', 'YJH', 'YJI', 'YJL', 'YJM', 'YJR', 'YJV', 'YJW', 'ZAZ', 'ZRC']


In [103]:
spain_airports = pd.DataFrame({'country_code':country_code_sp,
             'iata_code':spain_airpts_lst,
                              'name': name_airport})

In [104]:
spain_airports.to_sql('airports',engine, if_exists='append', index=False)

### France

In [22]:
france_airpts = get_data_airpts('FR')

<Response [200]>
2019-07-25 14:43:26.768482


In [105]:
france_airpts_lst,country_code_fr, name_airport = get_iata_codes(df_airpts=france_airpts)

In [106]:
france_airports = pd.DataFrame({'country_code':country_code_fr,
             'iata_code':france_airpts_lst,
                              'name': name_airport})

In [108]:
france_airports.to_sql('airports',engine, if_exists='append', index=False)

### Germany (DE)

In [24]:
germany_airpts = get_data_airpts('DE')

<Response [200]>
2019-07-25 14:43:29.140808


In [109]:
germany_airpts_lst,country_code_de, name_airport = get_iata_codes(df_airpts=germany_airpts)

In [110]:
germany_airports = pd.DataFrame({'country_code':country_code_de,
             'iata_code':germany_airpts_lst,
                              'name': name_airport})

In [111]:
germany_airports.to_sql('airports',engine, if_exists='append', index=False)

### Italy

In [26]:
italy_airpts = get_data_airpts('IT')

<Response [200]>
2019-07-25 14:43:31.721126


In [112]:
italy_airpts_lst, country_code_it, name_airport = get_iata_codes(df_airpts=italy_airpts)

In [113]:
italy_airports = pd.DataFrame({'country_code':country_code_it,
             'iata_code':italy_airpts_lst,
                              'name': name_airport})

In [114]:
italy_airports.to_sql('airports',engine, if_exists='append', index=False)

## All countries airports iata codes

In [29]:
countries_airpts_lst = spain_airpts_lst + france_airpts_lst + germany_airpts_lst + italy_airpts_lst

In [40]:
df_countries_airpt = pd.DataFrame(countries_airpts_lst)

## Wednesday, July 24

In [120]:
dict_airpt_df = {}

for aipt_code in countries_airpts_lst:
    
    df_airpt = get_data(airport = f"{aipt_code}")
    
    if type(df_airpt) == pd.core.frame.DataFrame:
        df_airpt_clean = clean_data(df_airpt)
        dict_airpt_df[aipt_code] = df_airpt_clean
        time.sleep(5)


<Response [200]>
2019-07-26 15:30:23.315689
<Response [200]>
2019-07-26 15:30:24.862241
<Response [200]>
2019-07-26 15:30:30.471620
<Response [200]>
2019-07-26 15:30:31.351418
<Response [200]>
2019-07-26 15:30:37.025934
<Response [200]>
2019-07-26 15:30:42.978515
<Response [200]>
2019-07-26 15:30:48.709055
<Response [200]>
2019-07-26 15:30:54.385854
<Response [200]>
2019-07-26 15:30:59.990070
<Response [200]>
2019-07-26 15:31:00.859631
<Response [200]>
2019-07-26 15:31:02.031965
<Response [200]>
2019-07-26 15:31:07.612053
<Response [200]>
2019-07-26 15:31:08.123521
<Response [200]>
2019-07-26 15:31:08.629241
<Response [200]>
2019-07-26 15:31:09.127291
<Response [200]>
2019-07-26 15:31:09.656265
<Response [200]>
2019-07-26 15:31:10.209342
<Response [200]>
2019-07-26 15:31:11.025971
<Response [200]>
2019-07-26 15:31:16.587189
<Response [200]>
2019-07-26 15:31:17.122835
<Response [200]>
2019-07-26 15:31:17.832510
<Response [200]>
2019-07-26 15:31:23.506852
<Response [200]>
2019-07-26 15:3

<Response [200]>
2019-07-26 15:37:33.203071
<Response [200]>
2019-07-26 15:37:34.176334
<Response [200]>
2019-07-26 15:37:39.778611
<Response [200]>
2019-07-26 15:37:41.127654
<Response [200]>
2019-07-26 15:37:42.079626
<Response [200]>
2019-07-26 15:37:47.771280
<Response [200]>
2019-07-26 15:37:48.645932
<Response [200]>
2019-07-26 15:37:49.567949
<Response [200]>
2019-07-26 15:37:50.596132
<Response [200]>
2019-07-26 15:37:56.423678
<Response [200]>
2019-07-26 15:38:02.149855
<Response [200]>
2019-07-26 15:38:02.742134
<Response [200]>
2019-07-26 15:38:03.337360
<Response [200]>
2019-07-26 15:38:03.904946
<Response [200]>
2019-07-26 15:38:05.041718
<Response [200]>
2019-07-26 15:38:11.174970
<Response [200]>
2019-07-26 15:38:12.342517
<Response [200]>
2019-07-26 15:38:13.342352
<Response [200]>
2019-07-26 15:38:14.199170
<Response [200]>
2019-07-26 15:38:15.417054
<Response [200]>
2019-07-26 15:38:16.458864
<Response [200]>
2019-07-26 15:38:17.154783
<Response [200]>
2019-07-26 15:3

<Response [200]>
2019-07-26 15:42:25.598562
<Response [200]>
2019-07-26 15:42:26.148629
<Response [200]>
2019-07-26 15:42:26.675179
<Response [200]>
2019-07-26 15:42:27.216267
<Response [200]>
2019-07-26 15:42:27.747598
<Response [200]>
2019-07-26 15:42:28.306995
<Response [200]>
2019-07-26 15:42:28.787926
<Response [200]>
2019-07-26 15:42:29.289759
<Response [200]>
2019-07-26 15:42:29.862742
<Response [200]>
2019-07-26 15:42:30.369119
<Response [200]>
2019-07-26 15:42:30.852818
<Response [200]>
2019-07-26 15:42:31.359519
<Response [200]>
2019-07-26 15:42:31.888175
<Response [200]>
2019-07-26 15:42:32.548591
<Response [200]>
2019-07-26 15:42:33.306108
<Response [200]>
2019-07-26 15:42:33.903527
<Response [200]>
2019-07-26 15:42:34.401621
<Response [200]>
2019-07-26 15:42:34.904488
<Response [200]>
2019-07-26 15:42:35.394373
<Response [200]>
2019-07-26 15:42:35.881435
<Response [200]>
2019-07-26 15:42:36.438896
<Response [200]>
2019-07-26 15:42:36.957282
<Response [200]>
2019-07-26 15:4

In [47]:
from sqlalchemy import create_engine


driver = 'mysql+pymysql:'
user = 'root'
password = '19693903'
ip = '127.0.0.1'
database = 'project_4'

In [48]:
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)


In [121]:
def save_to_sql_clean():
    for key, value in dict_airpt_df.items():
        
        value = value.astype({'schd_arri_time': 'datetime64[ns]','schd_depart_time':'datetime64[ns]'})
        value = value[['airln_name', 'airln_code','type', 'codeshared', 'status', 'icaonumb', 'flight','depart_terminal',\
      'depart_gate', 'depart_airpt' ,'schd_depart_time','depart_delay','arri_airpt','schd_arri_time',\
      'arri_delay']]
        value['depart_delay'] = value['depart_delay'].fillna(0)
        value['arri_delay'] = value['arri_delay'].fillna(0)
        value = value.astype({'depart_delay': int,'arri_delay':int})
        value = value.drop_duplicates(subset = ['flight','depart_terminal','schd_depart_time', 'depart_delay', \
                              'schd_arri_time', 'arri_delay'], keep='first')
        
        value.to_sql('flights',engine, if_exists='append', index=False)

In [122]:
save_to_sql_clean()